# Prompt Walkthrough

An overview of the different types of prompts in LangChain and how to use them

In [53]:
import yaml

with open("simple_prompt.yaml", "r") as stream:
    config = yaml.safe_load(stream)

In [54]:
config

{'input_variables': ['adjective', 'content'],
 'template': 'Tell me a {adjective} joke about {content}.\nLike what does it mean?'}

In [55]:
from pathlib import Path
from typing import Union
import yaml
def load_file(file: Union[str, Path]):
    if isinstance(file, str):
        file_path = Path(file)
    else:
        file_path = file
    if file_path.suffix == ".json":
        with open(file_path) as f:
            config = json.load(f)
    elif file_path.suffix == ".yaml":
        with open(file_path, "r") as f:
            config = yaml.safe_load(f)
    else:
        raise ValueError
    return load_prompt_from_config(config)
    

In [42]:
from langchain.prompts import Prompt, DynamicPrompt
import json
def load_prompt_from_config(config):
    if "type" in config:
        prompt_type = config.pop("type")
    else:
        prompt_type = "prompt"
    if prompt_type == "prompt":
        return _load_prompt(config)
    elif prompt_type == "dynamic_prompt":
        return _load_dynamic_prompt(config)
    else:
        raise ValueError
        
def _load_template(var_name: str, config: dict) -> dict:
    if f"{var_name}_path" in config:
        if var_name in config:
            raise ValueError(f"Both `{var_name}_path` and `{var_name}` cannot be provided.")
        template_path = Path(config.pop(f"{var_name}_path"))
        if template_path.suffix == ".txt":
            with open(template_path) as f:
                template = f.read()
        else:
            raise ValueError
        config[var_name] = template
    return config
        
        
def _load_dynamic_prompt(config):
    if "loader" in config:
        prompt_type = config.pop("loader")
    else:
        prompt_type = "init"
    if prompt_type == "init":
        config = _load_template("suffix", config)
        config = _load_template("prefix", config)
        return DynamicPrompt(**config)
    elif prompt_type == "from_structured_examples":
        config = _load_template("suffix", config)
        config = _load_template("prefix", config)
        config["example_prompt"] = _load_prompt(config["example_prompt"])
        if isinstance(config["examples"], list):
            pass
        elif isinstance(config["examples"], str):
            with open(config["examples"]) as f:
                examples = json.load(f)
            config["examples"] = examples
        else:
            raise ValueError
        return DynamicPrompt.from_structured_examples(**config)
    else:
        raise ValueError

def _load_prompt(config):
    if "loader" in config:
        prompt_type = config.pop("loader")
    else:
        prompt_type = "init"
    if prompt_type == "init":
        config = _load_template("template", config)
        return Prompt(**config)
    elif prompt_type == "from_examples":
        config = _load_template("suffix", config)
        config = _load_template("prefix", config)
        if isinstance(config["examples"], list):
            pass
        elif isinstance(config["examples"], str):
            with open(config["examples"]) as f:
                examples = json.load(f)
            config["examples"] = examples
        else:
            raise ValueError
        return Prompt.from_examples(**config)
    elif prompt_type == "from_structured_examples":
        config = _load_template("suffix", config)
        config = _load_template("prefix", config)
        config["example_prompt"] = _load_prompt(config["example_prompt"])
        if isinstance(config["examples"], list):
            pass
        elif isinstance(config["examples"], str):
            with open(config["examples"]) as f:
                examples = json.load(f)
            config["examples"] = examples
        else:
            raise ValueError
        return Prompt.from_structured_examples(**config)
    else:
        raise ValueError

### Basic Prompt

The most simple type of prompt - a string template that takes any number of input variables. The template should be formatted as a Python f-string.

In [4]:
# An example prompt with no input variables
_config = {
    "input_variables": [],
    "template": "Tell me a joke."
}
no_input_prompt = load_prompt_from_config(_config)
no_input_prompt.format()

'Tellme a joke.'

In [6]:
# An example prompt with one input variable
_config = {
    "input_variables": ["adjective"],
    "template": "Tell me a {adjective} joke."
}
one_input_prompt = load_prompt_from_config(_config)
one_input_prompt.format(adjective="funny")

'Tell me a funny joke.'

In [7]:
# An example prompt with multiple input variables
_config = {
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}
multiple_input_prompt = load_prompt_from_config(_config)
multiple_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [43]:
multiple_input_prompt = load_file("simple_prompt_with_template_file.json")
multiple_input_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### Examples
Examples are datapoints that can be used to show the model how to produce results. They can be either strings, or dictionaries that are then turned into strings by an example prompt itself.

In [12]:
string_examples = ["Input: happy\nOutput: sad", "Input: tall\nOutput: short"]
dict_examples = [{"input": "happy", "output": "sad"}, {"input": "tall", "output": "short"}]
example_prompt_config = {"input_variables": ["input","output"], "template": "Input: {input}\nOutput: {output}"}

### Simple Prompt with examples

We can then use these examples to construct prompts.

In [11]:
_config = {
    "loader": "from_examples",
    "examples": string_examples,
    "prefix": "Give the antonym of every input",
    "suffix": "Input: {adjective}\nOutput:", 
    "input_variables": ["adjective"],
}
prompt_from_string_examples = load_prompt_from_config(_config)
print(prompt_from_string_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [22]:
_config = {
    "loader": "from_examples",
    "examples": "string_examples.json",
    "prefix": "Give the antonym of every input",
    "suffix": "Input: {adjective}\nOutput:", 
    "input_variables": ["adjective"],
}
prompt_from_string_examples = load_prompt_from_config(_config)
print(prompt_from_string_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [17]:
_config = {
    "loader": "from_structured_examples",
    "examples": dict_examples,
    "example_prompt": example_prompt_config,
    "prefix": "Give the antonym of every input",
    "suffix": "Input: {adjective}\nOutput:", 
    "input_variables": ["adjective"],
}
prompt_from_structured_examples = load_prompt_from_config(_config)
print(prompt_from_structured_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [23]:
_config = {
    "loader": "from_structured_examples",
    "examples": "structured_examples.json",
    "example_prompt": example_prompt_config,
    "prefix": "Give the antonym of every input",
    "suffix": "Input: {adjective}\nOutput:", 
    "input_variables": ["adjective"],
}
prompt_from_structured_examples = load_prompt_from_config(_config)
print(prompt_from_structured_examples.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


### Dynamic Prompt

We also do more clever things with prompts - for example, only select a certain number of examples in order to limit the size of the text passed in. This will vary with the input text size.

In [29]:
_config = {
    "type": "dynamic_prompt",
    "loader": "from_structured_examples",
    "examples": "structured_examples.json",
    "example_prompt": example_prompt_config,
    "prefix": "Give the antonym of every input",
    "suffix": "Input: {adjective}\nOutput:", 
    "input_variables": ["adjective"],
    "max_length": 20,
}
dynamic_prompt = load_prompt_from_config(_config)

In [30]:
# An example with small input, so it selects both examples.
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: big
Output:


In [31]:
# An example with long input, so it selects only one example.
print(dynamic_prompt.format(adjective="big and huge and massive"))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive
Output:
